# 라이브러리 로드

In [1]:
# 기초 전처리
import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm import tqdm
%matplotlib inline

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import os

# 캔버스 사이즈 적용
plt.rcParams['figure.figsize'] = (12, 9)

# 컬럼 전체 확인 가능하도록 출력 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

##### 필요함수 로드

In [2]:
def load_data(path):
    df = pd.read_csv(path, encoding='CP949')
    df = df[ df['상권_구분_코드']=='A' ]  # 골목상권으로 테이블 제한
    
    return df

In [3]:
def categorization(x):
    """서비스 업종 코드명을 categorize하는 함수"""
    if x == '청과상' or x == '수산물판매' or x == '육류판매' or x == '미곡판매' or x == '반찬가게' or \
       x == '치과의원' or x == '의료기기' or x == '의약품' or x == '한의원' or x == '일반의원' or \
       x == '편의점' or x == '슈퍼마켓' or x == '세탁소' or x == '제과점':
        return '생활 필수'
    
    elif x == '미용재료' or x == '화장품' or x == '피부관리실' or x == '네일숍' or x == '미용실' or \
         x == '한복점' or x ==  '섬유제품' or x ==  '유아의류' or x == '안경' or x == '신발' or \
         x == '가방' or x == '의류임대' or x == '일반의류':
        return '미용/의류'
    
    elif x == '노래방' or x == '전자게임장' or x == 'PC방' or x == '골프연습장' or x == '당구장' or \
         x == 'DVD방' or x == '기타오락장' or x == '볼링장' or x == '스포츠클럽' or x == '스포츠 강습' or \
         x == '게스트하우스' or x == '여행사' or x == '서적':
        return '여가'
    
    elif x == '분식전문점' or x == '커피-음료' or x == '호프-간이주점' or x == '일식음식점' or \
         x == '중식음식점' or x == '한식음식점' or x == '양식음식점' or x == '패스트푸드점' or x == '치킨전문점':
        return '외식'
    
    elif x == '세무사사무소' or x == '법무사사무소' or x == '기타법무서비스' or x == '변리사사무소' or x == '회계사사무소' or \
         x == '변호사사무소' or x == '부동산중개업' or x == '독서실' or x == '문구' or x == '컴퓨터학원' or x == '고시원' or \
         x == '예술학원' or x == '외국어학원' or x == '일반교습학원' or x == '가전제품' or x == '핸드폰' or x == '통신기기수리' or \
         x == '컴퓨터및주변장치판매' or x == '가전제품수리' or x == '자동차미용' or x == '자동차수리' or x == '중고차판매' or \
         x == '자전거 및 기타운송장비' or x == '모터사이클수리'  or x == '주류도매':
        return '편의시설'
    
    else:
        return '기타'

# 2020 3분기 ~ 2021 2분기 총 매출 및 점포수
## 데이터 로드

In [4]:
df_base = pd.read_csv('../data/base_data/df_base.csv')
df_base.head()

,상권_코드
0,1000001
1,1000002
2,1000003
3,1000004
4,1000005


In [5]:
# 상권 점포
df_2021 = load_data('../data/row_data/서울시 우리마을가게 상권분석서비스(상권-추정매출).csv')
df_2021['서비스_업종_코드_명'] = df_2021['서비스_업종_코드_명'].apply(categorization)

df_2020 = load_data('../data/row_data/서울시 우리마을가게상권분석서비스(상권-추정매출)_2020.csv')
df_2020['서비스_업종_코드_명'] = df_2020['서비스_업종_코드_명'].apply(categorization)

df_2020.head(2)

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,주중_매출_비율,주말_매출_비율,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,시간대_00~06_매출_비율,시간대_06~11_매출_비율,시간대_11~14_매출_비율,시간대_14~17_매출_비율,시간대_17~21_매출_비율,시간대_21~24_매출_비율,남성_매출_비율,여성_매출_비율,연령대_10_매출_비율,연령대_20_매출_비율,연령대_30_매출_비율,연령대_40_매출_비율,연령대_50_매출_비율,연령대_60_이상_매출_비율,주중_매출_금액,주말_매출_금액,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,시간대_00~06_매출_금액,시간대_06~11_매출_금액,시간대_11~14_매출_금액,시간대_14~17_매출_금액,시간대_17~21_매출_금액,시간대_21~24_매출_금액,남성_매출_금액,여성_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,주중_매출_건수,주말_매출_건수,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,시간대_건수~06_매출_건수,시간대_건수~11_매출_건수,시간대_건수~14_매출_건수,시간대_건수~17_매출_건수,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
2,2020,1,A,골목상권,1000022,창신2길,CS200001,편의시설,22367209.0,82,100,0,0,71,29,0,0,0,0,0,0,0,0,100,0,71,29,0,0,0,100,0,0,22367209.0,0.0,0,13450049.0,8917160,0,0,0,0,0,0,0.0,0.0,22367209.0,0,12909615.0,9457594.0,0,0,0,22367209.0,0,0,82,0,0,54,28,0,0,0,0,0,0,0,0,82,0,54,28,0,0,0,82,0,0,2
4,2020,1,A,골목상권,1000001,계동길,CS100001,외식,431442455.0,18548,79,21,13,15,16,17,18,11,10,0,2,47,13,36,2,49,51,2,31,31,16,13,7,347666954.0,83775501.0,57836408,62676591.0,76100528,74583641,76469786,44469818,39305683,0,2948631,199264316.0,50651452.0,152694784.0,25883272,191019483.0,152066866.0,2544277,69650930,91286746,72418118.0,65658966,41527310,15115,3433,2750,2882,3268,3057,3158,1795,1638,0,639,9704,3342,4518,345,8752,7776,611,3828,4356,3428,2614,1690,19


## 점포 수 관련, 매출 Feature 테이블화

In [6]:
# 2020년 3/4분기로 테이블 축소
df_2020 = df_2020[   (df_2020['기준_분기_코드'] == 3)  | 
                     (df_2020['기준_분기_코드'] == 4)     ]
# 총 점포수 산출
store_2020 = df_2020.groupby(['상권_코드'])['점포수'].sum()
store_2021 = df_2021.groupby(['상권_코드'])['점포수'].sum()
store_total = store_2020 + store_2021

# 점포수 증가량
rate = store_2021 - store_2020
rate = rate.to_frame()

# 테이블 병합
df_full = pd.merge(df_base, store_total, on='상권_코드')
df_full = pd.merge(df_full, rate, on='상권_코드')

df_full.columns = ['상권_코드','점포수', '점포_증가량']
df_full.head()

,상권_코드,점포수,점포_증가량
0,1000001,429,-3
1,1000002,218,-12
2,1000003,728,-4
3,1000004,285,-9
4,1000005,152,2


# 개폐업 Feature

In [7]:
store = load_data('../data/row_data/서울시 우리마을가게 상권분석서비스(상권-점포).csv')
store = store[store['기준_년_코드']==2021]
store.head(3)

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,점포_수,유사_업종_점포_수,개업_율,개업_점포_수,폐업_률,폐업_점포_수,프랜차이즈_점포_수
29793,2021,3,A,골목상권,1001010,풍성로37가길,CS300043,전자상거래업,5,5,0,0,0,0,0
29794,2021,3,A,골목상권,1001010,풍성로37가길,CS300040,재생용품 판매점,0,0,0,0,0,0,0
29795,2021,3,A,골목상권,1001010,풍성로37가길,CS300036,조명용품,3,3,0,0,0,0,0


In [8]:
# 2021년 총 개업 점포수 및 폐업 점포수 도출
base_store = store.groupby('상권_코드')['개업_점포_수','폐업_점포_수'].sum().reset_index()
base_store['개폐업'] = 0
base_store['개폐업_비율'] = 0

# 파생변수 생성
for idx in range(base_store.shape[0]):
    base_store['개폐업'][idx] = base_store['개업_점포_수'][idx] - base_store['폐업_점포_수'][idx]
    if base_store['개업_점포_수'][idx] == 0:
        base_store['개폐업_비율'][idx] == 0
    else:
        base_store['개폐업_비율'][idx] = base_store['폐업_점포_수'][idx] / base_store['개업_점포_수'][idx]
        
# 개업점포수 및 폐업점포수가 0인 경우
base_store['개폐업_비율'] = base_store['개폐업_비율'].fillna(0)  
base_store.head()

,상권_코드,개업_점포_수,폐업_점포_수,개폐업,개폐업_비율
0,1000001,21,13,8,0
1,1000002,3,6,-3,2
2,1000003,11,21,-10,1
3,1000004,7,5,2,0
4,1000005,2,2,0,1


In [9]:
df_full = pd.merge(df_full, base_store, on='상권_코드')
df_full.head()

,상권_코드,점포수,점포_증가량,개업_점포_수,폐업_점포_수,개폐업,개폐업_비율
0,1000001,429,-3,21,13,8,0
1,1000002,218,-12,3,6,-3,2
2,1000003,728,-4,11,21,-10,1
3,1000004,285,-9,7,5,2,0
4,1000005,152,2,2,2,0,1


# 점포 업종당 Feature 생성

In [10]:
# 서울시 빅데이터 캠퍼스 반출 자료이기 때문에 공개하지 않습니다.
origin_df = pd.read_csv('../data/row_data/매출액_데이터.csv')
income_df = origin_df.iloc[:,20:50]

columns = []
for col in income_df.columns:
    columns.append(col[7:])

income_df.columns = columns
income_df = pd.concat([origin_df['index'], income_df], axis=1)

In [11]:
income_df

,index,가전/가구,교육용품,기타요식,기타유통,기타의료,미용서비스,백화점,병원,서비스,숙박,스포츠/문화/레저,스포츠/문화/레저용품,약국,여행,유아교육,유흥,음/식료품,의복/의류,인테리어,일식/중식/양식,자동차서비스/용품,자동차판매,제과/커피/패스트푸드,주유,패션/잡화,편의점,학원,한식,할인점/슈퍼마켓,화장품
0,1000001,1.724337e+07,1.064328e+07,1.141504e+09,0.000000e+00,0.000000e+00,1.866668e+07,0.000000e+00,1.098479e+08,6.991791e+07,2.383339e+07,7.042669e+08,2.794810e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.987585e+07,1.629542e+08,3.375704e+07,9.667553e+08,2.162874e+07,0.000000e+00,1.229597e+08,0.000000e+00,1.324405e+08,4.037474e+08,1.738525e+08,1.169807e+09,2.902993e+08,0.000000e+00
1,1000002,2.848917e+07,0.000000e+00,4.280148e+07,0.000000e+00,0.000000e+00,3.986462e+07,0.000000e+00,0.000000e+00,5.863048e+07,8.298050e+06,2.257537e+07,3.091750e+07,0.000000e+00,0.000000e+00,0.000000e+00,2.493700e+07,1.510012e+07,8.445374e+08,1.351476e+08,0.000000e+00,9.276520e+06,7.218960e+06,7.321844e+07,0.000000e+00,3.398267e+08,5.464145e+08,0.000000e+00,1.664063e+08,1.004982e+06,9.117890e+06
2,1000003,8.318360e+07,5.094446e+09,5.385404e+08,0.000000e+00,3.048000e+06,2.689181e+07,0.000000e+00,0.000000e+00,1.292482e+08,2.149619e+08,2.505250e+07,1.607377e+08,0.000000e+00,0.000000e+00,0.000000e+00,5.380292e+07,1.741951e+08,1.155379e+08,3.013530e+07,5.699978e+08,9.421422e+07,0.000000e+00,1.946193e+08,0.000000e+00,4.734395e+08,5.577831e+08,5.166470e+07,7.857958e+08,3.979991e+07,0.000000e+00
3,1000004,7.169375e+06,5.960735e+07,1.786134e+08,0.000000e+00,0.000000e+00,2.508195e+07,0.000000e+00,1.265900e+08,1.711343e+08,0.000000e+00,1.109184e+07,2.000630e+06,3.534005e+07,0.000000e+00,0.000000e+00,2.845950e+06,4.845681e+07,0.000000e+00,3.778561e+07,1.061836e+08,1.963030e+07,0.000000e+00,1.311017e+08,0.000000e+00,1.320800e+05,4.353733e+08,0.000000e+00,6.325940e+08,4.756951e+08,0.000000e+00
4,1000005,8.582460e+05,0.000000e+00,1.962510e+09,4.380559e+07,0.000000e+00,2.163800e+06,0.000000e+00,0.000000e+00,2.945062e+08,0.000000e+00,6.327800e+07,1.631586e+08,0.000000e+00,0.000000e+00,1.105550e+07,0.000000e+00,1.159619e+07,1.062179e+08,2.851677e+07,6.854519e+08,2.144353e+08,0.000000e+00,4.501143e+08,1.312884e+09,1.273297e+07,3.702061e+08,7.090619e+07,1.556898e+09,3.016168e+07,9.443500e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487,1001491,1.576457e+08,1.330123e+06,7.420877e+09,5.590185e+07,0.000000e+00,6.088020e+08,0.000000e+00,1.173635e+09,2.463513e+08,4.047628e+08,9.549674e+08,1.861592e+09,0.000000e+00,0.000000e+00,0.000000e+00,4.257763e+07,4.246356e+08,4.148411e+09,1.729255e+08,8.858908e+09,3.226795e+08,2.079901e+09,7.625452e+08,0.000000e+00,1.255536e+09,1.197991e+09,5.680695e+07,3.830366e+09,7.135465e+08,1.421341e+09
1488,1001492,2.966044e+09,2.212273e+08,1.624485e+10,1.017313e+08,5.404666e+08,1.638166e+09,2.172624e+11,1.168295e+10,2.666638e+09,2.125099e+09,2.291789e+09,3.451150e+09,5.499865e+09,3.025947e+08,8.202600e+05,6.776614e+07,2.573021e+09,6.050112e+09,1.263539e+09,1.280162e+10,2.001653e+09,0.000000e+00,4.491829e+09,3.283558e+08,8.458168e+09,5.005794e+09,1.663806e+08,4.661540e+10,9.712533e+08,1.871787e+09
1489,1001493,2.232087e+08,7.616646e+07,2.684104e+09,7.887226e+07,0.000000e+00,2.872129e+08,0.000000e+00,7.647130e+09,5.442839e+08,3.328121e+08,8.096151e+08,1.478253e+09,3.227650e+08,0.000000e+00,0.000000e+00,1.619213e+07,4.970184e+08,3.912528e+09,4.639011e+08,9.086506e+08,1.132954e+09,4.451830e+07,1.075357e+09,0.000000e+00,9.476181e+08,2.753384e+09,4.188869e+07,3.453470e+09,1.196464e+09,2.015556e+08
1490,1001494,9.156332e+08,4.214434e+08,4.503911e+09,5.911123e+07,3.317071e+08,3.344051e+08,0.000000e+00,3.154040e+09,7.655337e+08,1.502542e+08,3.304011e+10,8.279684e+08,1.044492e+09,4.203265e+06,0.000000e+00,2.790357e+08,2.168512e+08,3.135285e+08,7.876124e+08,4.734673e+09,5.053303e+07,0.000000e+00,2.312281e+09,0.000000e+00,1.354325e+09,1.417830e+09,2.583957e+09,1.181805e+10,1.670175e+08,3.871660e+08


In [12]:
new_table = pd.DataFrame()

new_table['상권_코드'] = income_df['index']
new_table['유흥_총_매출액'] = income_df['유흥'] + income_df['숙박']
new_table['패션/미용_총_매출액'] = income_df['미용서비스'] + income_df['화장품'] + income_df['패션/잡화'] + income_df['의복/의류']
new_table['교육_총_매출액'] = income_df['유아교육'] + income_df['학원'] + income_df['교육용품']
new_table['요식_총_매출액'] = income_df['기타요식'] + income_df['한식'] + income_df['일식/중식/양식'] + income_df['제과/커피/패스트푸드']
new_table['의료_총_매출액'] = income_df['기타의료'] + income_df['병원'] + income_df['약국']
new_table['자동차_총_매출액'] = income_df['자동차서비스/용품'] + income_df['자동차판매'] + income_df['주유']
new_table['기타_총_매출액'] = income_df['가전/가구'] + income_df['인테리어'] + income_df['여행'] + income_df['서비스'] + income_df['스포츠/문화/레저'] + income_df['백화점'] + income_df['스포츠/문화/레저용품']
new_table['유통업_총_매출액'] = income_df['기타유통'] + income_df['음/식료품'] + income_df['편의점'] + income_df['할인점/슈퍼마켓']

new_table = new_table.set_index('상권_코드')
new_table['합계'] = new_table.sum(axis=1)
new_table.head()

,유흥_총_매출액,패션/미용_총_매출액,교육_총_매출액,요식_총_매출액,의료_총_매출액,자동차_총_매출액,기타_총_매출액,유통업_총_매출액,합계
상권_코드,,,,,,,,,
1000001,2.383339e+07,3.140614e+08,1.844958e+08,3.401026e+09,1.098479e+08,2.162874e+07,1.104666e+09,7.539225e+08,5.913482e+09
1000002,3.323505e+07,1.233347e+09,0.000000e+00,2.824262e+08,0.000000e+00,1.649548e+07,2.757601e+08,5.625196e+08,2.403783e+09
1000003,2.687648e+08,6.158692e+08,5.146111e+09,2.088953e+09,3.048000e+06,9.421422e+07,4.283574e+08,7.717781e+08,9.417096e+09
1000004,2.845950e+06,2.521403e+07,5.960735e+07,1.048493e+09,1.619301e+08,1.963030e+07,2.291818e+08,9.595252e+08,2.506427e+09
1000005,0.000000e+00,1.305581e+08,8.196169e+07,4.654974e+09,0.000000e+00,1.527319e+09,5.503179e+08,4.557695e+08,7.400900e+09


In [13]:
cols = new_table.columns[:-1]
for col in cols:
    new_table[str(col)[:-1]+'_비율'] = new_table[col] / new_table['합계']
    
new_table.drop(cols, axis=1, inplace=True)
new_table.drop('합계', axis=1, inplace=True)
new_table.head()

,유흥_총_매출_비율,패션/미용_총_매출_비율,교육_총_매출_비율,요식_총_매출_비율,의료_총_매출_비율,자동차_총_매출_비율,기타_총_매출_비율,유통업_총_매출_비율
상권_코드,,,,,,,,
1000001,0.004030,0.053109,0.031199,0.575131,0.018576,0.003658,0.186805,0.127492
1000002,0.013826,0.513086,0.000000,0.117492,0.000000,0.006862,0.114719,0.234014
1000003,0.028540,0.065399,0.546465,0.221826,0.000324,0.010005,0.045487,0.081955
1000004,0.001135,0.010060,0.023782,0.418322,0.064606,0.007832,0.091438,0.382826
1000005,0.000000,0.017641,0.011075,0.628974,0.000000,0.206369,0.074358,0.061583


In [14]:
df_full = pd.merge(df_full, new_table, left_on='상권_코드', right_index=True)
df_full.head()

,상권_코드,점포수,점포_증가량,개업_점포_수,폐업_점포_수,개폐업,개폐업_비율,유흥_총_매출_비율,패션/미용_총_매출_비율,교육_총_매출_비율,요식_총_매출_비율,의료_총_매출_비율,자동차_총_매출_비율,기타_총_매출_비율,유통업_총_매출_비율
0,1000001,429,-3,21,13,8,0,0.004030,0.053109,0.031199,0.575131,0.018576,0.003658,0.186805,0.127492
1,1000002,218,-12,3,6,-3,2,0.013826,0.513086,0.000000,0.117492,0.000000,0.006862,0.114719,0.234014
2,1000003,728,-4,11,21,-10,1,0.028540,0.065399,0.546465,0.221826,0.000324,0.010005,0.045487,0.081955
3,1000004,285,-9,7,5,2,0,0.001135,0.010060,0.023782,0.418322,0.064606,0.007832,0.091438,0.382826
4,1000005,152,2,2,2,0,1,0.000000,0.017641,0.011075,0.628974,0.000000,0.206369,0.074358,0.061583


In [15]:
# 업종당 점포수 산출
service_2020 = df_2020.groupby(['상권_코드', '서비스_업종_코드_명']).size()
service_2021 = df_2021.groupby(['상권_코드', '서비스_업종_코드_명']).size()

# 테이블 재 구조화
service_2020 = service_2020.unstack().fillna(0)
service_2021 = service_2021.unstack().fillna(0)

# 1년 업종당 점포수 산출
total_service = service_2020 + service_2021

# 테이블 재 구조화
total_service = total_service.reset_index()
total_service.columns = ['상권_코드', '기타_총_점포수',
                         '미용/의류_총_점포수','생활_필수_총_점포수',
                         '여가_총_점포수','외식_총_점포수','편의시설_총_점포수']
total_service.head()

,상권_코드,기타_총_점포수,미용/의류_총_점포수,생활_필수_총_점포수,여가_총_점포수,외식_총_점포수,편의시설_총_점포수
0,1000001,4.0,12.0,16.0,1.0,28.0,4.0
1,1000002,8.0,11.0,12.0,4.0,12.0,1.0
2,1000003,15.0,10.0,23.0,12.0,32.0,9.0
3,1000004,0.0,0.0,32.0,2.0,29.0,4.0
4,1000005,4.0,5.0,8.0,4.0,20.0,4.0


In [16]:
# 외식, 미용/의류 컬럼만 추출
select_col = total_service[['상권_코드','외식_총_점포수','미용/의류_총_점포수']]
df_full = pd.merge(df_full, select_col, on='상권_코드')

df_full.head()

,상권_코드,점포수,점포_증가량,개업_점포_수,폐업_점포_수,개폐업,개폐업_비율,유흥_총_매출_비율,패션/미용_총_매출_비율,교육_총_매출_비율,요식_총_매출_비율,의료_총_매출_비율,자동차_총_매출_비율,기타_총_매출_비율,유통업_총_매출_비율,외식_총_점포수,미용/의류_총_점포수
0,1000001,429,-3,21,13,8,0,0.004030,0.053109,0.031199,0.575131,0.018576,0.003658,0.186805,0.127492,28.0,12.0
1,1000002,218,-12,3,6,-3,2,0.013826,0.513086,0.000000,0.117492,0.000000,0.006862,0.114719,0.234014,12.0,11.0
2,1000003,728,-4,11,21,-10,1,0.028540,0.065399,0.546465,0.221826,0.000324,0.010005,0.045487,0.081955,32.0,10.0
3,1000004,285,-9,7,5,2,0,0.001135,0.010060,0.023782,0.418322,0.064606,0.007832,0.091438,0.382826,29.0,0.0
4,1000005,152,2,2,2,0,1,0.000000,0.017641,0.011075,0.628974,0.000000,0.206369,0.074358,0.061583,20.0,5.0


# 면적당 Feature 생성

In [17]:
area = gpd.read_file('../data/geo/상권_geometry_좌표.geojson', driver='GeoJSON')
area= area[area['TRDAR_SE_C'] == 'A']

# geometry 기반의 면적 컬럼 생성
area['area'] = area.geometry.area
area = area[['TRDAR_CD', 'area']]
area = area.sort_values('TRDAR_CD').reset_index(drop=True)
area.columns = ['상권_코드','면적']
area['면적'] = area['면적'].agg(lambda x : x * 100000 )
area['상권_코드'] = area['상권_코드'].astype(int)
area.head()

,상권_코드,면적
0,1000001,1.278637
1,1000002,0.323278
2,1000003,1.160775
3,1000004,0.449096
4,1000005,3.744966


In [18]:
df_full = pd.merge(df_full, area, on='상권_코드')
df_full.head()

,상권_코드,점포수,점포_증가량,개업_점포_수,폐업_점포_수,개폐업,개폐업_비율,유흥_총_매출_비율,패션/미용_총_매출_비율,교육_총_매출_비율,요식_총_매출_비율,의료_총_매출_비율,자동차_총_매출_비율,기타_총_매출_비율,유통업_총_매출_비율,외식_총_점포수,미용/의류_총_점포수,면적
0,1000001,429,-3,21,13,8,0,0.004030,0.053109,0.031199,0.575131,0.018576,0.003658,0.186805,0.127492,28.0,12.0,1.278637
1,1000002,218,-12,3,6,-3,2,0.013826,0.513086,0.000000,0.117492,0.000000,0.006862,0.114719,0.234014,12.0,11.0,0.323278
2,1000003,728,-4,11,21,-10,1,0.028540,0.065399,0.546465,0.221826,0.000324,0.010005,0.045487,0.081955,32.0,10.0,1.160775
3,1000004,285,-9,7,5,2,0,0.001135,0.010060,0.023782,0.418322,0.064606,0.007832,0.091438,0.382826,29.0,0.0,0.449096
4,1000005,152,2,2,2,0,1,0.000000,0.017641,0.011075,0.628974,0.000000,0.206369,0.074358,0.061583,20.0,5.0,3.744966


In [19]:
# 면적당 점포수
df_full['면적당_점포수'] = df_full['점포수']/df_full['면적']

# 면적당 미용의류 점포수
df_full['면적당_미용의류_점포수'] = df_full['미용/의류_총_점포수']/df_full['면적']

# 면적당 외식 점포수
df_full['면적당_외식_점포수'] = df_full['외식_총_점포수']/df_full['면적']
df_full.head()

,상권_코드,점포수,점포_증가량,개업_점포_수,폐업_점포_수,개폐업,개폐업_비율,유흥_총_매출_비율,패션/미용_총_매출_비율,교육_총_매출_비율,요식_총_매출_비율,의료_총_매출_비율,자동차_총_매출_비율,기타_총_매출_비율,유통업_총_매출_비율,외식_총_점포수,미용/의류_총_점포수,면적,면적당_점포수,면적당_미용의류_점포수,면적당_외식_점포수
0,1000001,429,-3,21,13,8,0,0.004030,0.053109,0.031199,0.575131,0.018576,0.003658,0.186805,0.127492,28.0,12.0,1.278637,335.513594,9.384996,21.898323
1,1000002,218,-12,3,6,-3,2,0.013826,0.513086,0.000000,0.117492,0.000000,0.006862,0.114719,0.234014,12.0,11.0,0.323278,674.342105,34.026437,37.119749
2,1000003,728,-4,11,21,-10,1,0.028540,0.065399,0.546465,0.221826,0.000324,0.010005,0.045487,0.081955,32.0,10.0,1.160775,627.167417,8.614937,27.567799
3,1000004,285,-9,7,5,2,0,0.001135,0.010060,0.023782,0.418322,0.064606,0.007832,0.091438,0.382826,29.0,0.0,0.449096,634.607631,0.000000,64.574110
4,1000005,152,2,2,2,0,1,0.000000,0.017641,0.011075,0.628974,0.000000,0.206369,0.074358,0.061583,20.0,5.0,3.744966,40.587820,1.335126,5.340503


# 요일별/ 성별 매출 Feature

In [20]:
day_col = ['월요일_매출_금액', '화요일_매출_금액', 
           '수요일_매출_금액', '목요일_매출_금액', 
           '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']

day_2020 = df_2020.groupby(['상권_코드'])[day_col].sum()
day_2021 = df_2021.groupby(['상권_코드'])[day_col].sum()

day_total = day_2020 + day_2021
day_total = day_total.reset_index()

day_total = day_total.set_index('상권_코드')
day_total['합계'] = day_total.sum(axis=1)

for col in day_col:
    day_total[str(col)[:-2]+'비율'] = day_total[col] / day_total['합계']
    
day_total.drop(day_col, axis=1, inplace=True)
day_total.drop('합계', axis=1, inplace=True)

df_full = pd.merge(df_full, day_total, left_on='상권_코드', right_index=True)
df_full.head()

,상권_코드,점포수,점포_증가량,개업_점포_수,폐업_점포_수,개폐업,개폐업_비율,유흥_총_매출_비율,패션/미용_총_매출_비율,교육_총_매출_비율,요식_총_매출_비율,의료_총_매출_비율,자동차_총_매출_비율,기타_총_매출_비율,유통업_총_매출_비율,외식_총_점포수,미용/의류_총_점포수,면적,면적당_점포수,면적당_미용의류_점포수,면적당_외식_점포수,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율
0,1000001,429,-3,21,13,8,0,0.004030,0.053109,0.031199,0.575131,0.018576,0.003658,0.186805,0.127492,28.0,12.0,1.278637,335.513594,9.384996,21.898323,0.118590,0.131493,0.148730,0.132410,0.154490,0.183464,0.130823
1,1000002,218,-12,3,6,-3,2,0.013826,0.513086,0.000000,0.117492,0.000000,0.006862,0.114719,0.234014,12.0,11.0,0.323278,674.342105,34.026437,37.119749,0.116593,0.116357,0.126294,0.125684,0.156763,0.188164,0.170143
2,1000003,728,-4,11,21,-10,1,0.028540,0.065399,0.546465,0.221826,0.000324,0.010005,0.045487,0.081955,32.0,10.0,1.160775,627.167417,8.614937,27.567799,0.144447,0.144302,0.142621,0.122779,0.143191,0.211421,0.091239
3,1000004,285,-9,7,5,2,0,0.001135,0.010060,0.023782,0.418322,0.064606,0.007832,0.091438,0.382826,29.0,0.0,0.449096,634.607631,0.000000,64.574110,0.156436,0.160766,0.167509,0.139482,0.157609,0.123508,0.094691
4,1000005,152,2,2,2,0,1,0.000000,0.017641,0.011075,0.628974,0.000000,0.206369,0.074358,0.061583,20.0,5.0,3.744966,40.587820,1.335126,5.340503,0.080943,0.118395,0.140430,0.113667,0.141985,0.217003,0.187577


In [21]:
gender_col = ['남성_매출_금액', '여성_매출_금액']

gender_2020 = df_2020.groupby(['상권_코드'])[gender_col].sum()
gender_2021 = df_2021.groupby(['상권_코드'])[gender_col].sum()

gender_total = gender_2020 + gender_2021
gender_total['총합'] = gender_total.sum(axis=1)

for col in gender_col:
    gender_total[str(col)[:-2]+'비율'] = gender_total[col] / gender_total['총합']
    
gender_total.drop(gender_col, axis=1, inplace=True)
gender_total.drop('총합', axis=1, inplace=True)

df_full = pd.merge(df_full, gender_total, left_on='상권_코드', right_index=True)
df_full.head()

,상권_코드,점포수,점포_증가량,개업_점포_수,폐업_점포_수,개폐업,개폐업_비율,유흥_총_매출_비율,패션/미용_총_매출_비율,교육_총_매출_비율,요식_총_매출_비율,의료_총_매출_비율,자동차_총_매출_비율,기타_총_매출_비율,유통업_총_매출_비율,외식_총_점포수,미용/의류_총_점포수,면적,면적당_점포수,면적당_미용의류_점포수,면적당_외식_점포수,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,남성_매출_비율,여성_매출_비율
0,1000001,429,-3,21,13,8,0,0.004030,0.053109,0.031199,0.575131,0.018576,0.003658,0.186805,0.127492,28.0,12.0,1.278637,335.513594,9.384996,21.898323,0.118590,0.131493,0.148730,0.132410,0.154490,0.183464,0.130823,0.386854,0.613146
1,1000002,218,-12,3,6,-3,2,0.013826,0.513086,0.000000,0.117492,0.000000,0.006862,0.114719,0.234014,12.0,11.0,0.323278,674.342105,34.026437,37.119749,0.116593,0.116357,0.126294,0.125684,0.156763,0.188164,0.170143,0.401264,0.598736
2,1000003,728,-4,11,21,-10,1,0.028540,0.065399,0.546465,0.221826,0.000324,0.010005,0.045487,0.081955,32.0,10.0,1.160775,627.167417,8.614937,27.567799,0.144447,0.144302,0.142621,0.122779,0.143191,0.211421,0.091239,0.579238,0.420762
3,1000004,285,-9,7,5,2,0,0.001135,0.010060,0.023782,0.418322,0.064606,0.007832,0.091438,0.382826,29.0,0.0,0.449096,634.607631,0.000000,64.574110,0.156436,0.160766,0.167509,0.139482,0.157609,0.123508,0.094691,0.549805,0.450195
4,1000005,152,2,2,2,0,1,0.000000,0.017641,0.011075,0.628974,0.000000,0.206369,0.074358,0.061583,20.0,5.0,3.744966,40.587820,1.335126,5.340503,0.080943,0.118395,0.140430,0.113667,0.141985,0.217003,0.187577,0.481882,0.518118


# 상권 배후지 인구 Feature
## 데이터 로드

In [22]:
popul = load_data('../data/row_data/서울시 우리마을가게 상권분석서비스(상권배후지-생활인구).csv')
popul.head(2)

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,남성_생활인구_수,여성_생활인구_수,연령대_10_생활인구_수,연령대_20_생활인구_수,연령대_30_생활인구_수,연령대_40_생활인구_수,연령대_50_생활인구_수,연령대_60_이상_생활인구_수,시간대_1_생활인구_수,시간대_2_생활인구_수,시간대_3_생활인구_수,시간대_4_생활인구_수,시간대_5_생활인구_수,시간대_6_생활인구_수,월요일_생활인구_수,화요일_생활인구_수,수요일_생활인구_수,목요일_생활인구_수,금요일_생활인구_수,토요일_생활인구_수,일요일_생활인구_수,남성연령대_10_월요일시간대_1_생활인구_수,남성연령대_10_월요일시간대_2_생활인구_수,남성연령대_10_월요일시간대_3_생활인구_수,남성연령대_10_월요일시간대_4_생활인구_수,남성연령대_10_월요일시간대_5_생활인구_수,남성연령대_10_월요일시간대_6_생활인구_수,남성연령대_10_화요일시간대_1_생활인구_수,남성연령대_10_화요일시간대_2_생활인구_수,남성연령대_10_화요일시간대_3_생활인구_수,남성연령대_10_화요일시간대_4_생활인구_수,남성연령대_10_화요일시간대_5_생활인구_수,남성연령대_10_화요일시간대_6_생활인구_수,남성연령대_10_수요일시간대_1_생활인구_수,남성연령대_10_수요일시간대_2_생활인구_수,남성연령대_10_수요일시간대_3_생활인구_수,남성연령대_10_수요일시간대_4_생활인구_수,남성연령대_10_수요일시간대_5_생활인구_수,남성연령대_10_수요일시간대_6_생활인구_수,남성연령대_10_목요일시간대_1_생활인구_수,남성연령대_10_목요일시간대_2_생활인구_수,남성연령대_10_목요일시간대_3_생활인구_수,남성연령대_10_목요일시간대_4_생활인구_수,남성연령대_10_목요일시간대_5_생활인구_수,남성연령대_10_목요일시간대_6_생활인구_수,남성연령대_10_금요일시간대_1_생활인구_수,남성연령대_10_금요일시간대_2_생활인구_수,남성연령대_10_금요일시간대_3_생활인구_수,남성연령대_10_금요일시간대_4_생활인구_수,남성연령대_10_금요일시간대_5_생활인구_수,남성연령대_10_금요일시간대_6_생활인구_수,남성연령대_10_토요일시간대_1_생활인구_수,남성연령대_10_토요일시간대_2_생활인구_수,남성연령대_10_토요일시간대_3_생활인구_수,남성연령대_10_토요일시간대_4_생활인구_수,남성연령대_10_토요일시간대_5_생활인구_수,남성연령대_10_토요일시간대_6_생활인구_수,남성연령대_10_일요일시간대_1_생활인구_수,남성연령대_10_일요일시간대_2_생활인구_수,남성연령대_10_일요일시간대_3_생활인구_수,남성연령대_10_일요일시간대_4_생활인구_수,남성연령대_10_일요일시간대_5_생활인구_수,남성연령대_10_일요일시간대_6_생활인구_수,남성연령대_20_월요일시간대_1_생활인구_수,남성연령대_20_월요일시간대_2_생활인구_수,남성연령대_20_월요일시간대_3_생활인구_수,남성연령대_20_월요일시간대_4_생활인구_수,남성연령대_20_월요일시간대_5_생활인구_수,남성연령대_20_월요일시간대_6_생활인구_수,남성연령대_20_화요일시간대_1_생활인구_수,남성연령대_20_화요일시간대_2_생활인구_수,남성연령대_20_화요일시간대_3_생활인구_수,남성연령대_20_화요일시간대_4_생활인구_수,남성연령대_20_화요일시간대_5_생활인구_수,남성연령대_20_화요일시간대_6_생활인구_수,남성연령대_20_수요일시간대_1_생활인구_수,남성연령대_20_수요일시간대_2_생활인구_수,남성연령대_20_수요일시간대_3_생활인구_수,남성연령대_20_수요일시간대_4_생활인구_수,남성연령대_20_수요일시간대_5_생활인구_수,남성연령대_20_수요일시간대_6_생활인구_수,남성연령대_20_목요일시간대_1_생활인구_수,남성연령대_20_목요일시간대_2_생활인구_수,남성연령대_20_목요일시간대_3_생활인구_수,남성연령대_20_목요일시간대_4_생활인구_수,남성연령대_20_목요일시간대_5_생활인구_수,남성연령대_20_목요일시간대_6_생활인구_수,남성연령대_20_금요일시간대_1_생활인구_수,남성연령대_20_금요일시간대_2_생활인구_수,남성연령대_20_금요일시간대_3_생활인구_수,남성연령대_20_금요일시간대_4_생활인구_수,남성연령대_20_금요일시간대_5_생활인구_수,남성연령대_20_금요일시간대_6_생활인구_수,남성연령대_20_토요일시간대_1_생활인구_수,남성연령대_20_토요일시간대_2_생활인구_수,남성연령대_20_토요일시간대_3_생활인구_수,남성연령대_20_토요일시간대_4_생활인구_수,남성연령대_20_토요일시간대_5_생활인구_수,남성연령대_20_토요일시간대_6_생활인구_수,남성연령대_20_일요일시간대_1_생활인구_수,남성연령대_20_일요일시간대_2_생활인구_수,남성연령대_20_일요일시간대_3_생활인구_수,남성연령대_20_일요일시간대_4_생활인구_수,남성연령대_20_일요일시간대_5_생활인구_수,남성연령대_20_일요일시간대_6_생활인구_수,남성연령대_30_월요일시간대_1_생활인구_수,남성연령대_30_월요일시간대_2_생활인구_수,남성연령대_30_월요일시간대_3_생활인구_수,남성연령대_30_월요일시간대_4_생활인구_수,남성연령대_30_월요일시간대_5_생활인구_수,남성연령대_30_월요일시간대_6_생활인구_수,남성연령대_30_화요일시간대_1_생활인구_수,남성연령대_30_화요일시간대_2_생활인구_수,남성연령대_30_화요일시간대_3_생활인구_수,남성연령대_30_화요일시간대_4_생활인구_수,남성연령대_30_화요일시간대_5_생활인구_수,남성연령대_30_화요일시간대_6_생활인구_수,남성연령대_30_수요일시간대_1_생활인구_수,남성연령대_30_수요일시간대_2_생활인구_수,남성연령대_30_수요일시간대_3_생활인구_수,남성연령대_30_수요일시간대_4_생활인구_수,남성연령대_30_수요일시간대_5_생활인구_수,남성연령대_30_수요일시간대_6_생활인구_수,남성연령대_30_목요일시간대_1_생활인구_수,남성연령대_30_목요일시간대_2_생활인구_수,남성연령대_30_목요일시간대_3_생활인구_수,남성연령대_30_목요일시간대_4_생활인구_수,남성연령대_30_목요일시간대_5_생활인구_수,남성연령대_30_목요일시간대_6_생활인구_수,남성연령대_30_금요일시간대_1_생활인구_수,남성연령대_30_금요일시간대_2_생활인구_수,남성연령대_30_금요일시간대_3_생활인구_수,남성연령대_30_금요일시간대_4_생활인구_수,남성연령대_30_금요일시간대_5_생활인구_수,남성연령대_30_금요일시간대_6_생활인구_수,남성연령대_30_토요일시간대_1_생활인구_수,남성연령대_30_토요일시간대_2_생활인구_수,남성연령대_30_토요일시간대_3_생활인구_수,남성연령대_30_토요일시간대_4_생활인구_수,남성연령대_30_토요일시간대_5_생활인구_수,남성연령대_30_토요일시간대_6_생활인구_수,남성연령대_30_일요일시간대_1_생활인구_수,남성연령대_30_일요일시간대_2_생활인구_수,남성연령대_30_일요일시간대_3_생활인구_수,남성연령대_30_일요일시간대_4_생활인구_수,남성연령대_30_일요일시간대_5_생활인구_수,남성연령대_30_일요일시간대_6_생활인구_수,남성연령대_40_월요일시간대_1_생활인구_수,남성연령대_40_월요일시간대_2_생활인구_수,남성연령대_40_월요일시간대_3_생활인구_수,남성연령대_40_월요일시간대_4_생활인구_수,남성연령대_40_월요일시간대_5_생활인구_수,남성연령대_40_월요일시간대_6_생활인구_수,남성연령대_40_화요일시간대_1_생활인구_수,남성연령대_40_화요일시간대_2_생활인구_수,남성연령대_40_화요일시간대_3_생활인구_수,남성연령대_40_화요일시간대_4_생활인구_수,남성연령대_40_화요일시간대_5_생활인구_수,남성연령대_40_화요일시간대_6_생활인구_수,남성연령대_40_수요일시간대_1_생활인구_수,남성연령대_40_수요일시간대_2_생활인구_수,남성연령대_40_수요일시간대_3_생활인구_수,남성연령대_40_수요일시간대_4_생활인구_수,남성연령대_40_수요일시간대_5_생활인구_수,남성연령대_40_수요일시간대_6_생활인구_수,남성연령대_40_목요일시간대_1_생활인구_수,남성연령대_40_목요일시간대_2_생활인구_수,남성연령대_40_목요일시간대_3_생활인구_수,남성

In [23]:
# 2021년 1/2분기 및 2020년 3/4 분기로 테이블 제한
popul = popul[ (popul['기준_년_코드']==2021) | (popul['기준_년_코드']==2020) ]
popul = popul[ 
           ((popul['기준_년_코드']==2020) & (popul['기준_분기_코드'] == 3)) | 
           ((popul['기준_년_코드']==2020) & (popul['기준_분기_코드'] == 4)) |
           (popul['기준_년_코드']==2021)         
         ]

print('2020년 분기 unique 값: ', popul[popul['기준_년_코드']==2020]['기준_분기_코드'].unique())
print('2021년 분기 unique 값: ', popul[popul['기준_년_코드']==2021]['기준_분기_코드'].unique())
popul.head(2)

2020년 분기 unique 값:  [4 3]
2021년 분기 unique 값:  [2 1]


,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_생활인구_수,남성_생활인구_수,여성_생활인구_수,연령대_10_생활인구_수,연령대_20_생활인구_수,연령대_30_생활인구_수,연령대_40_생활인구_수,연령대_50_생활인구_수,연령대_60_이상_생활인구_수,시간대_1_생활인구_수,시간대_2_생활인구_수,시간대_3_생활인구_수,시간대_4_생활인구_수,시간대_5_생활인구_수,시간대_6_생활인구_수,월요일_생활인구_수,화요일_생활인구_수,수요일_생활인구_수,목요일_생활인구_수,금요일_생활인구_수,토요일_생활인구_수,일요일_생활인구_수,남성연령대_10_월요일시간대_1_생활인구_수,남성연령대_10_월요일시간대_2_생활인구_수,남성연령대_10_월요일시간대_3_생활인구_수,남성연령대_10_월요일시간대_4_생활인구_수,남성연령대_10_월요일시간대_5_생활인구_수,남성연령대_10_월요일시간대_6_생활인구_수,남성연령대_10_화요일시간대_1_생활인구_수,남성연령대_10_화요일시간대_2_생활인구_수,남성연령대_10_화요일시간대_3_생활인구_수,남성연령대_10_화요일시간대_4_생활인구_수,남성연령대_10_화요일시간대_5_생활인구_수,남성연령대_10_화요일시간대_6_생활인구_수,남성연령대_10_수요일시간대_1_생활인구_수,남성연령대_10_수요일시간대_2_생활인구_수,남성연령대_10_수요일시간대_3_생활인구_수,남성연령대_10_수요일시간대_4_생활인구_수,남성연령대_10_수요일시간대_5_생활인구_수,남성연령대_10_수요일시간대_6_생활인구_수,남성연령대_10_목요일시간대_1_생활인구_수,남성연령대_10_목요일시간대_2_생활인구_수,남성연령대_10_목요일시간대_3_생활인구_수,남성연령대_10_목요일시간대_4_생활인구_수,남성연령대_10_목요일시간대_5_생활인구_수,남성연령대_10_목요일시간대_6_생활인구_수,남성연령대_10_금요일시간대_1_생활인구_수,남성연령대_10_금요일시간대_2_생활인구_수,남성연령대_10_금요일시간대_3_생활인구_수,남성연령대_10_금요일시간대_4_생활인구_수,남성연령대_10_금요일시간대_5_생활인구_수,남성연령대_10_금요일시간대_6_생활인구_수,남성연령대_10_토요일시간대_1_생활인구_수,남성연령대_10_토요일시간대_2_생활인구_수,남성연령대_10_토요일시간대_3_생활인구_수,남성연령대_10_토요일시간대_4_생활인구_수,남성연령대_10_토요일시간대_5_생활인구_수,남성연령대_10_토요일시간대_6_생활인구_수,남성연령대_10_일요일시간대_1_생활인구_수,남성연령대_10_일요일시간대_2_생활인구_수,남성연령대_10_일요일시간대_3_생활인구_수,남성연령대_10_일요일시간대_4_생활인구_수,남성연령대_10_일요일시간대_5_생활인구_수,남성연령대_10_일요일시간대_6_생활인구_수,남성연령대_20_월요일시간대_1_생활인구_수,남성연령대_20_월요일시간대_2_생활인구_수,남성연령대_20_월요일시간대_3_생활인구_수,남성연령대_20_월요일시간대_4_생활인구_수,남성연령대_20_월요일시간대_5_생활인구_수,남성연령대_20_월요일시간대_6_생활인구_수,남성연령대_20_화요일시간대_1_생활인구_수,남성연령대_20_화요일시간대_2_생활인구_수,남성연령대_20_화요일시간대_3_생활인구_수,남성연령대_20_화요일시간대_4_생활인구_수,남성연령대_20_화요일시간대_5_생활인구_수,남성연령대_20_화요일시간대_6_생활인구_수,남성연령대_20_수요일시간대_1_생활인구_수,남성연령대_20_수요일시간대_2_생활인구_수,남성연령대_20_수요일시간대_3_생활인구_수,남성연령대_20_수요일시간대_4_생활인구_수,남성연령대_20_수요일시간대_5_생활인구_수,남성연령대_20_수요일시간대_6_생활인구_수,남성연령대_20_목요일시간대_1_생활인구_수,남성연령대_20_목요일시간대_2_생활인구_수,남성연령대_20_목요일시간대_3_생활인구_수,남성연령대_20_목요일시간대_4_생활인구_수,남성연령대_20_목요일시간대_5_생활인구_수,남성연령대_20_목요일시간대_6_생활인구_수,남성연령대_20_금요일시간대_1_생활인구_수,남성연령대_20_금요일시간대_2_생활인구_수,남성연령대_20_금요일시간대_3_생활인구_수,남성연령대_20_금요일시간대_4_생활인구_수,남성연령대_20_금요일시간대_5_생활인구_수,남성연령대_20_금요일시간대_6_생활인구_수,남성연령대_20_토요일시간대_1_생활인구_수,남성연령대_20_토요일시간대_2_생활인구_수,남성연령대_20_토요일시간대_3_생활인구_수,남성연령대_20_토요일시간대_4_생활인구_수,남성연령대_20_토요일시간대_5_생활인구_수,남성연령대_20_토요일시간대_6_생활인구_수,남성연령대_20_일요일시간대_1_생활인구_수,남성연령대_20_일요일시간대_2_생활인구_수,남성연령대_20_일요일시간대_3_생활인구_수,남성연령대_20_일요일시간대_4_생활인구_수,남성연령대_20_일요일시간대_5_생활인구_수,남성연령대_20_일요일시간대_6_생활인구_수,남성연령대_30_월요일시간대_1_생활인구_수,남성연령대_30_월요일시간대_2_생활인구_수,남성연령대_30_월요일시간대_3_생활인구_수,남성연령대_30_월요일시간대_4_생활인구_수,남성연령대_30_월요일시간대_5_생활인구_수,남성연령대_30_월요일시간대_6_생활인구_수,남성연령대_30_화요일시간대_1_생활인구_수,남성연령대_30_화요일시간대_2_생활인구_수,남성연령대_30_화요일시간대_3_생활인구_수,남성연령대_30_화요일시간대_4_생활인구_수,남성연령대_30_화요일시간대_5_생활인구_수,남성연령대_30_화요일시간대_6_생활인구_수,남성연령대_30_수요일시간대_1_생활인구_수,남성연령대_30_수요일시간대_2_생활인구_수,남성연령대_30_수요일시간대_3_생활인구_수,남성연령대_30_수요일시간대_4_생활인구_수,남성연령대_30_수요일시간대_5_생활인구_수,남성연령대_30_수요일시간대_6_생활인구_수,남성연령대_30_목요일시간대_1_생활인구_수,남성연령대_30_목요일시간대_2_생활인구_수,남성연령대_30_목요일시간대_3_생활인구_수,남성연령대_30_목요일시간대_4_생활인구_수,남성연령대_30_목요일시간대_5_생활인구_수,남성연령대_30_목요일시간대_6_생활인구_수,남성연령대_30_금요일시간대_1_생활인구_수,남성연령대_30_금요일시간대_2_생활인구_수,남성연령대_30_금요일시간대_3_생활인구_수,남성연령대_30_금요일시간대_4_생활인구_수,남성연령대_30_금요일시간대_5_생활인구_수,남성연령대_30_금요일시간대_6_생활인구_수,남성연령대_30_토요일시간대_1_생활인구_수,남성연령대_30_토요일시간대_2_생활인구_수,남성연령대_30_토요일시간대_3_생활인구_수,남성연령대_30_토요일시간대_4_생활인구_수,남성연령대_30_토요일시간대_5_생활인구_수,남성연령대_30_토요일시간대_6_생활인구_수,남성연령대_30_일요일시간대_1_생활인구_수,남성연령대_30_일요일시간대_2_생활인구_수,남성연령대_30_일요일시간대_3_생활인구_수,남성연령대_30_일요일시간대_4_생활인구_수,남성연령대_30_일요일시간대_5_생활인구_수,남성연령대_30_일요일시간대_6_생활인구_수,남성연령대_40_월요일시간대_1_생활인구_수,남성연령대_40_월요일시간대_2_생활인구_수,남성연령대_40_월요일시간대_3_생활인구_수,남성연령대_40_월요일시간대_4_생활인구_수,남성연령대_40_월요일시간대_5_생활인구_수,남성연령대_40_월요일시간대_6_생활인구_수,남성연령대_40_화요일시간대_1_생활인구_수,남성연령대_40_화요일시간대_2_생활인구_수,남성연령대_40_화요일시간대_3_생활인구_수,남성연령대_40_화요일시간대_4_생활인구_수,남성연령대_40_화요일시간대_5_생활인구_수,남성연령대_40_화요일시간대_6_생활인구_수,남성연령대_40_수요일시간대_1_생활인구_수,남성연령대_40_수요일시간대_2_생활인구_수,남성연령대_40_수요일시간대_3_생활인구_수,남성연령대_40_수요일시간대_4_생활인구_수,남성연령대_40_수요일시간대_5_생활인구_수,남성연령대_40_수요일시간대_6_생활인구_수,남성연령대_40_목요일시간대_1_생활인구_수,남성연령대_40_목요일시간대_2_생활인구_수,남성연령대_40_목요일시간대_3_생활인구_수,남성

In [24]:
col_list = popul.iloc[:,6:28].columns

df_popul = popul.groupby(['상권_코드'])[col_list].sum().reset_index()
df_popul = pd.merge(df_base, df_popul)
print(df_popul.shape)
df_popul.head()

(1009, 23)


,상권_코드,총_생활인구_수,남성_생활인구_수,여성_생활인구_수,연령대_10_생활인구_수,연령대_20_생활인구_수,연령대_30_생활인구_수,연령대_40_생활인구_수,연령대_50_생활인구_수,연령대_60_이상_생활인구_수,시간대_1_생활인구_수,시간대_2_생활인구_수,시간대_3_생활인구_수,시간대_4_생활인구_수,시간대_5_생활인구_수,시간대_6_생활인구_수,월요일_생활인구_수,화요일_생활인구_수,수요일_생활인구_수,목요일_생활인구_수,금요일_생활인구_수,토요일_생활인구_수,일요일_생활인구_수
0,1000001,4324568,2193021,2131546,539326,613811,830622,871492,666560,802756,784753,966918,737816,731799,700663,402616,648404,671523,663182,661833,647096,530394,502137
1,1000002,14572603,7302651,7269951,1139824,2395368,2778982,2495751,2420125,3342559,2987914,3058515,2246600,2253728,2449248,1576598,2100487,2113428,2114383,2104128,2088132,2027654,2024391
2,1000003,6931076,3966981,2964095,346596,1290132,1298139,1276708,1135201,1584302,904485,1347371,1311840,1383582,1411510,572292,1039460,1085002,1079350,1072242,1058901,872185,723940
3,1000004,12659811,5925747,6734063,2163623,3794991,1379547,1648944,1389617,2283089,3889185,2756008,1390214,1270606,1725169,1628625,1821418,1831914,1822679,1820578,1780184,1767680,1815363
4,1000005,1840860,865152,975711,269838,212859,216631,325791,312262,503478,496022,401417,219594,210490,277669,235668,257712,260156,262067,259023,261080,268340,272482


In [25]:
cols = df_popul.columns[2:]

for col in cols:
    df_popul[str(col)[:-2]+'_비율'] = df_popul[col] / df_popul['총_생활인구_수']
    
df_popul.drop(cols, axis=1, inplace=True)
df_popul.drop('총_생활인구_수', axis=1, inplace=True)
df_popul.head()

,상권_코드,남성_생활인구_비율,여성_생활인구_비율,연령대_10_생활인구_비율,연령대_20_생활인구_비율,연령대_30_생활인구_비율,연령대_40_생활인구_비율,연령대_50_생활인구_비율,연령대_60_이상_생활인구_비율,시간대_1_생활인구_비율,시간대_2_생활인구_비율,시간대_3_생활인구_비율,시간대_4_생활인구_비율,시간대_5_생활인구_비율,시간대_6_생활인구_비율,월요일_생활인구_비율,화요일_생활인구_비율,수요일_생활인구_비율,목요일_생활인구_비율,금요일_생활인구_비율,토요일_생활인구_비율,일요일_생활인구_비율
0,1000001,0.507108,0.492892,0.124712,0.141936,0.192071,0.201521,0.154133,0.185627,0.181464,0.223587,0.170610,0.169219,0.162019,0.093100,0.149935,0.155281,0.153352,0.153040,0.149633,0.122647,0.116113
1,1000002,0.501122,0.498878,0.078217,0.164375,0.190699,0.171263,0.166074,0.229373,0.205036,0.209881,0.154166,0.154655,0.168072,0.108189,0.144139,0.145027,0.145093,0.144389,0.143292,0.139142,0.138918
2,1000003,0.572347,0.427653,0.050006,0.186137,0.187293,0.184201,0.163784,0.228580,0.130497,0.194396,0.189269,0.199620,0.203649,0.082569,0.149971,0.156542,0.155726,0.154701,0.152776,0.125837,0.104448
3,1000004,0.468075,0.531924,0.170905,0.299767,0.108971,0.130250,0.109766,0.180341,0.307207,0.217697,0.109813,0.100365,0.136271,0.128645,0.143874,0.144703,0.143974,0.143808,0.140617,0.139629,0.143396
4,1000005,0.469972,0.530030,0.146583,0.115630,0.117679,0.176978,0.169628,0.273502,0.269451,0.218059,0.119289,0.114343,0.150837,0.128021,0.139995,0.141323,0.142361,0.140708,0.141825,0.145769,0.148019


In [26]:
df_full = pd.merge(df_full, df_popul, on='상권_코드')
print(df_full.shape)
df_full.head()

(1009, 51)


,상권_코드,점포수,점포_증가량,개업_점포_수,폐업_점포_수,개폐업,개폐업_비율,유흥_총_매출_비율,패션/미용_총_매출_비율,교육_총_매출_비율,요식_총_매출_비율,의료_총_매출_비율,자동차_총_매출_비율,기타_총_매출_비율,유통업_총_매출_비율,외식_총_점포수,미용/의류_총_점포수,면적,면적당_점포수,면적당_미용의류_점포수,면적당_외식_점포수,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,남성_매출_비율,여성_매출_비율,남성_생활인구_비율,여성_생활인구_비율,연령대_10_생활인구_비율,연령대_20_생활인구_비율,연령대_30_생활인구_비율,연령대_40_생활인구_비율,연령대_50_생활인구_비율,연령대_60_이상_생활인구_비율,시간대_1_생활인구_비율,시간대_2_생활인구_비율,시간대_3_생활인구_비율,시간대_4_생활인구_비율,시간대_5_생활인구_비율,시간대_6_생활인구_비율,월요일_생활인구_비율,화요일_생활인구_비율,수요일_생활인구_비율,목요일_생활인구_비율,금요일_생활인구_비율,토요일_생활인구_비율,일요일_생활인구_비율
0,1000001,429,-3,21,13,8,0,0.004030,0.053109,0.031199,0.575131,0.018576,0.003658,0.186805,0.127492,28.0,12.0,1.278637,335.513594,9.384996,21.898323,0.118590,0.131493,0.148730,0.132410,0.154490,0.183464,0.130823,0.386854,0.613146,0.507108,0.492892,0.124712,0.141936,0.192071,0.201521,0.154133,0.185627,0.181464,0.223587,0.170610,0.169219,0.162019,0.093100,0.149935,0.155281,0.153352,0.153040,0.149633,0.122647,0.116113
1,1000002,218,-12,3,6,-3,2,0.013826,0.513086,0.000000,0.117492,0.000000,0.006862,0.114719,0.234014,12.0,11.0,0.323278,674.342105,34.026437,37.119749,0.116593,0.116357,0.126294,0.125684,0.156763,0.188164,0.170143,0.401264,0.598736,0.501122,0.498878,0.078217,0.164375,0.190699,0.171263,0.166074,0.229373,0.205036,0.209881,0.154166,0.154655,0.168072,0.108189,0.144139,0.145027,0.145093,0.144389,0.143292,0.139142,0.138918
2,1000003,728,-4,11,21,-10,1,0.028540,0.065399,0.546465,0.221826,0.000324,0.010005,0.045487,0.081955,32.0,10.0,1.160775,627.167417,8.614937,27.567799,0.144447,0.144302,0.142621,0.122779,0.143191,0.211421,0.091239,0.579238,0.420762,0.572347,0.427653,0.050006,0.186137,0.187293,0.184201,0.163784,0.228580,0.130497,0.194396,0.189269,0.199620,0.203649,0.082569,0.149971,0.156542,0.155726,0.154701,0.152776,0.125837,0.104448
3,1000004,285,-9,7,5,2,0,0.001135,0.010060,0.023782,0.418322,0.064606,0.007832,0.091438,0.382826,29.0,0.0,0.449096,634.607631,0.000000,64.574110,0.156436,0.160766,0.167509,0.139482,0.157609,0.123508,0.094691,0.549805,0.450195,0.468075,0.531924,0.170905,0.299767,0.108971,0.130250,0.109766,0.180341,0.307207,0.217697,0.109813,0.100365,0.136271,0.128645,0.143874,0.144703,0.143974,0.143808,0.140617,0.139629,0.143396
4,1000005,152,2,2,2,0,1,0.000000,0.017641,0.011075,0.628974,0.000000,0.206369,0.074358,0.061583,20.0,5.0,3.744966,40.587820,1.335126,5.340503,0.080943,0.118395,0.140430,0.113667,0.141985,0.217003,0.187577,0.481882,0.518118,0.469972,0.530030,0.146583,0.115630,0.117679,0.176978,0.169628,0.273502,0.269451,0.218059,0.119289,0.114343,0.150837,0.128021,0.139995,0.141323,0.142361,0.140708,0.141825,0.145769,0.148019


# 직장 인구 Feature

In [27]:
df = load_data('../data/row_data/서울시 우리마을가게 상권분석서비스(상권배후지-직장인구).csv')
df.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,연령대_10_직장_인구_수,연령대_20_직장_인구_수,연령대_30_직장_인구_수,연령대_40_직장_인구_수,연령대_50_직장_인구_수,연령대_60_이상_직장_인구_수,남성연령대_10_직장_인구_수,남성연령대_20_직장_인구_수,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
0,2021,2,A,골목상권,1001010,풍성로37가길,2748,1678,1070,6,410,594,684,640,414,2,264,372,393,389,258,4,146,222,291,251,156
1,2021,2,A,골목상권,1001009,천호옛길,6064,3635,2429,9,803,1428,1488,1645,691,5,447,723,830,1088,542,4,356,705,658,557,149
2,2021,2,A,골목상권,1001008,천호옛14길,2503,1310,1193,8,459,534,594,535,373,2,250,263,287,285,223,6,209,271,307,250,150
3,2021,2,A,골목상권,1001007,천호대로219길,966,662,304,0,107,187,231,268,173,0,53,131,167,183,128,0,54,56,64,85,45
4,2021,2,A,골목상권,1001006,천호대로197길,2122,1190,932,2,260,374,420,580,486,2,147,232,225,279,305,0,113,142,195,301,181


In [28]:
기준년_2020_3분기 = (df['기준_년_코드'] == 2020) & (df['기준_분기_코드'] == 3)
기준년_2020_4분기 = (df['기준_년_코드'] == 2020) & (df['기준_분기_코드'] == 4)
기준년_2021_1분기 = (df['기준_년_코드'] == 2021) & (df['기준_분기_코드'] == 1)
기준년_2021_2분기 = (df['기준_년_코드'] == 2021) & (df['기준_분기_코드'] == 2)

df = df[  기준년_2020_3분기  |  기준년_2020_4분기  |  기준년_2021_1분기  |  기준년_2021_2분기  ]
print(df.기준_년_코드.unique())
print(df.기준_분기_코드.unique())

df.drop(['기준_년_코드', '기준_분기_코드','상권_구분_코드','상권_구분_코드_명','상권_코드_명'], axis=1, inplace=True)
df.head()

[2021 2020]
[2 1 4 3]


,상권_코드,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,연령대_10_직장_인구_수,연령대_20_직장_인구_수,연령대_30_직장_인구_수,연령대_40_직장_인구_수,연령대_50_직장_인구_수,연령대_60_이상_직장_인구_수,남성연령대_10_직장_인구_수,남성연령대_20_직장_인구_수,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
0,1001010,2748,1678,1070,6,410,594,684,640,414,2,264,372,393,389,258,4,146,222,291,251,156
1,1001009,6064,3635,2429,9,803,1428,1488,1645,691,5,447,723,830,1088,542,4,356,705,658,557,149
2,1001008,2503,1310,1193,8,459,534,594,535,373,2,250,263,287,285,223,6,209,271,307,250,150
3,1001007,966,662,304,0,107,187,231,268,173,0,53,131,167,183,128,0,54,56,64,85,45
4,1001006,2122,1190,932,2,260,374,420,580,486,2,147,232,225,279,305,0,113,142,195,301,181


In [29]:
popul = df.groupby(['상권_코드']).sum()

for col in popul.columns[1:22]:
    popul[col] = popul[col] / popul['총_직장_인구_수']
    
popul = popul.iloc[:, :3]
popul.head()

,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수
상권_코드,,,
1000001,54800,0.742482,0.257518
1000002,47092,0.534103,0.465897
1000003,158024,0.662709,0.337291
1000004,6784,0.564269,0.435731
1000005,17008,0.581373,0.418627


In [30]:
df_full = pd.merge(df_full, popul, on='상권_코드')
print(df_full.shape)
df_full.head()

(1009, 54)


,상권_코드,점포수,점포_증가량,개업_점포_수,폐업_점포_수,개폐업,개폐업_비율,유흥_총_매출_비율,패션/미용_총_매출_비율,교육_총_매출_비율,요식_총_매출_비율,의료_총_매출_비율,자동차_총_매출_비율,기타_총_매출_비율,유통업_총_매출_비율,외식_총_점포수,미용/의류_총_점포수,면적,면적당_점포수,면적당_미용의류_점포수,면적당_외식_점포수,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,남성_매출_비율,여성_매출_비율,남성_생활인구_비율,여성_생활인구_비율,연령대_10_생활인구_비율,연령대_20_생활인구_비율,연령대_30_생활인구_비율,연령대_40_생활인구_비율,연령대_50_생활인구_비율,연령대_60_이상_생활인구_비율,시간대_1_생활인구_비율,시간대_2_생활인구_비율,시간대_3_생활인구_비율,시간대_4_생활인구_비율,시간대_5_생활인구_비율,시간대_6_생활인구_비율,월요일_생활인구_비율,화요일_생활인구_비율,수요일_생활인구_비율,목요일_생활인구_비율,금요일_생활인구_비율,토요일_생활인구_비율,일요일_생활인구_비율,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수
0,1000001,429,-3,21,13,8,0,0.004030,0.053109,0.031199,0.575131,0.018576,0.003658,0.186805,0.127492,28.0,12.0,1.278637,335.513594,9.384996,21.898323,0.118590,0.131493,0.148730,0.132410,0.154490,0.183464,0.130823,0.386854,0.613146,0.507108,0.492892,0.124712,0.141936,0.192071,0.201521,0.154133,0.185627,0.181464,0.223587,0.170610,0.169219,0.162019,0.093100,0.149935,0.155281,0.153352,0.153040,0.149633,0.122647,0.116113,54800,0.742482,0.257518
1,1000002,218,-12,3,6,-3,2,0.013826,0.513086,0.000000,0.117492,0.000000,0.006862,0.114719,0.234014,12.0,11.0,0.323278,674.342105,34.026437,37.119749,0.116593,0.116357,0.126294,0.125684,0.156763,0.188164,0.170143,0.401264,0.598736,0.501122,0.498878,0.078217,0.164375,0.190699,0.171263,0.166074,0.229373,0.205036,0.209881,0.154166,0.154655,0.168072,0.108189,0.144139,0.145027,0.145093,0.144389,0.143292,0.139142,0.138918,47092,0.534103,0.465897
2,1000003,728,-4,11,21,-10,1,0.028540,0.065399,0.546465,0.221826,0.000324,0.010005,0.045487,0.081955,32.0,10.0,1.160775,627.167417,8.614937,27.567799,0.144447,0.144302,0.142621,0.122779,0.143191,0.211421,0.091239,0.579238,0.420762,0.572347,0.427653,0.050006,0.186137,0.187293,0.184201,0.163784,0.228580,0.130497,0.194396,0.189269,0.199620,0.203649,0.082569,0.149971,0.156542,0.155726,0.154701,0.152776,0.125837,0.104448,158024,0.662709,0.337291
3,1000004,285,-9,7,5,2,0,0.001135,0.010060,0.023782,0.418322,0.064606,0.007832,0.091438,0.382826,29.0,0.0,0.449096,634.607631,0.000000,64.574110,0.156436,0.160766,0.167509,0.139482,0.157609,0.123508,0.094691,0.549805,0.450195,0.468075,0.531924,0.170905,0.299767,0.108971,0.130250,0.109766,0.180341,0.307207,0.217697,0.109813,0.100365,0.136271,0.128645,0.143874,0.144703,0.143974,0.143808,0.140617,0.139629,0.143396,6784,0.564269,0.435731
4,1000005,152,2,2,2,0,1,0.000000,0.017641,0.011075,0.628974,0.000000,0.206369,0.074358,0.061583,20.0,5.0,3.744966,40.587820,1.335126,5.340503,0.080943,0.118395,0.140430,0.113667,0.141985,0.217003,0.187577,0.481882,0.518118,0.469972,0.530030,0.146583,0.115630,0.117679,0.176978,0.169628,0.273502,0.269451,0.218059,0.119289,0.114343,0.150837,0.128021,0.139995,0.141323,0.142361,0.140708,0.141825,0.145769,0.148019,17008,0.581373,0.418627


# 상권 소득소비 관련 Feature

In [31]:
df = load_data('../data/row_data/서울시 우리마을가게 상권분석서비스(상권배후지-소득소비).csv')
df.head()

,기준 년 코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액
0,2021,2,A,골목상권,1001010,풍성로37가길,2850735,6,12120296344,3292343150,1466629217,892634747,1607315609,1972205430,444384439,377360177,1550457530,516966045
1,2021,2,A,골목상권,1001009,천호옛길,3146942,7,8976116500,2526811041,1073907318,659101424,1225903534,1395888645,327727539,283116450,1109489033,374171516
2,2021,2,A,골목상권,1001008,천호옛14길,2774214,6,6175425196,1846295541,733624837,464028324,882017929,906576672,219745655,205715662,662629513,254791063
3,2021,2,A,골목상권,1001007,천호대로219길,2561943,6,1609692543,452889712,190742939,108296299,224205264,256333051,52791935,47847994,208823957,67761392
4,2021,2,A,골목상권,1001006,천호대로197길,3001401,7,10437230074,2691268507,1267558670,747339442,1326410044,1722932932,403612797,304016442,1535009509,439081731


In [32]:
기준년_2020_3분기 = (df['기준 년 코드'] == 2020) & (df['기준_분기_코드'] == 3)
기준년_2020_4분기 = (df['기준 년 코드'] == 2020) & (df['기준_분기_코드'] == 4)
기준년_2021_1분기 = (df['기준 년 코드'] == 2021) & (df['기준_분기_코드'] == 1)
기준년_2021_2분기 = (df['기준 년 코드'] == 2021) & (df['기준_분기_코드'] == 2)

df = df[  기준년_2020_3분기  |  기준년_2020_4분기  |  기준년_2021_1분기  |  기준년_2021_2분기  ]
print(df['기준 년 코드'].unique())
print(df['기준_분기_코드'].unique())

df.drop(['기준 년 코드', '기준_분기_코드','상권_구분_코드','상권_구분_코드_명','상권_코드_명', '소득_구간_코드', '지출_총금액'], axis=1, inplace=True)
df.head()

[2021 2020]
[2 1 4 3]


,상권_코드,월_평균_소득_금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액
0,1001010,2850735,3292343150,1466629217,892634747,1607315609,1972205430,444384439,377360177,1550457530,516966045
1,1001009,3146942,2526811041,1073907318,659101424,1225903534,1395888645,327727539,283116450,1109489033,374171516
2,1001008,2774214,1846295541,733624837,464028324,882017929,906576672,219745655,205715662,662629513,254791063
3,1001007,2561943,452889712,190742939,108296299,224205264,256333051,52791935,47847994,208823957,67761392
4,1001006,3001401,2691268507,1267558670,747339442,1326410044,1722932932,403612797,304016442,1535009509,439081731


In [33]:
income = df.groupby('상권_코드')['월_평균_소득_금액'].mean().to_frame()
income.head()

,월_평균_소득_금액
상권_코드,
1000001,3528699
1000002,3283059
1000003,2909305
1000004,2951442
1000005,3977676


In [34]:
df_full = pd.merge(df_full, income, left_on='상권_코드', right_index=True)
print(df_full.shape)
df_full.head()

(1009, 55)


,상권_코드,점포수,점포_증가량,개업_점포_수,폐업_점포_수,개폐업,개폐업_비율,유흥_총_매출_비율,패션/미용_총_매출_비율,교육_총_매출_비율,요식_총_매출_비율,의료_총_매출_비율,자동차_총_매출_비율,기타_총_매출_비율,유통업_총_매출_비율,외식_총_점포수,미용/의류_총_점포수,면적,면적당_점포수,면적당_미용의류_점포수,면적당_외식_점포수,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,남성_매출_비율,여성_매출_비율,남성_생활인구_비율,여성_생활인구_비율,연령대_10_생활인구_비율,연령대_20_생활인구_비율,연령대_30_생활인구_비율,연령대_40_생활인구_비율,연령대_50_생활인구_비율,연령대_60_이상_생활인구_비율,시간대_1_생활인구_비율,시간대_2_생활인구_비율,시간대_3_생활인구_비율,시간대_4_생활인구_비율,시간대_5_생활인구_비율,시간대_6_생활인구_비율,월요일_생활인구_비율,화요일_생활인구_비율,수요일_생활인구_비율,목요일_생활인구_비율,금요일_생활인구_비율,토요일_생활인구_비율,일요일_생활인구_비율,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,월_평균_소득_금액
0,1000001,429,-3,21,13,8,0,0.004030,0.053109,0.031199,0.575131,0.018576,0.003658,0.186805,0.127492,28.0,12.0,1.278637,335.513594,9.384996,21.898323,0.118590,0.131493,0.148730,0.132410,0.154490,0.183464,0.130823,0.386854,0.613146,0.507108,0.492892,0.124712,0.141936,0.192071,0.201521,0.154133,0.185627,0.181464,0.223587,0.170610,0.169219,0.162019,0.093100,0.149935,0.155281,0.153352,0.153040,0.149633,0.122647,0.116113,54800,0.742482,0.257518,3528699
1,1000002,218,-12,3,6,-3,2,0.013826,0.513086,0.000000,0.117492,0.000000,0.006862,0.114719,0.234014,12.0,11.0,0.323278,674.342105,34.026437,37.119749,0.116593,0.116357,0.126294,0.125684,0.156763,0.188164,0.170143,0.401264,0.598736,0.501122,0.498878,0.078217,0.164375,0.190699,0.171263,0.166074,0.229373,0.205036,0.209881,0.154166,0.154655,0.168072,0.108189,0.144139,0.145027,0.145093,0.144389,0.143292,0.139142,0.138918,47092,0.534103,0.465897,3283059
2,1000003,728,-4,11,21,-10,1,0.028540,0.065399,0.546465,0.221826,0.000324,0.010005,0.045487,0.081955,32.0,10.0,1.160775,627.167417,8.614937,27.567799,0.144447,0.144302,0.142621,0.122779,0.143191,0.211421,0.091239,0.579238,0.420762,0.572347,0.427653,0.050006,0.186137,0.187293,0.184201,0.163784,0.228580,0.130497,0.194396,0.189269,0.199620,0.203649,0.082569,0.149971,0.156542,0.155726,0.154701,0.152776,0.125837,0.104448,158024,0.662709,0.337291,2909305
3,1000004,285,-9,7,5,2,0,0.001135,0.010060,0.023782,0.418322,0.064606,0.007832,0.091438,0.382826,29.0,0.0,0.449096,634.607631,0.000000,64.574110,0.156436,0.160766,0.167509,0.139482,0.157609,0.123508,0.094691,0.549805,0.450195,0.468075,0.531924,0.170905,0.299767,0.108971,0.130250,0.109766,0.180341,0.307207,0.217697,0.109813,0.100365,0.136271,0.128645,0.143874,0.144703,0.143974,0.143808,0.140617,0.139629,0.143396,6784,0.564269,0.435731,2951442
4,1000005,152,2,2,2,0,1,0.000000,0.017641,0.011075,0.628974,0.000000,0.206369,0.074358,0.061583,20.0,5.0,3.744966,40.587820,1.335126,5.340503,0.080943,0.118395,0.140430,0.113667,0.141985,0.217003,0.187577,0.481882,0.518118,0.469972,0.530030,0.146583,0.115630,0.117679,0.176978,0.169628,0.273502,0.269451,0.218059,0.119289,0.114343,0.150837,0.128021,0.139995,0.141323,0.142361,0.140708,0.141825,0.145769,0.148019,17008,0.581373,0.418627,3977676


# 최종 테이블 저장

In [35]:
df_full.to_csv('../data/indicator/preprocess_dev.csv', index=False)
pd.read_csv('../data/indicator/preprocess_dev.csv').head()

,상권_코드,점포수,점포_증가량,개업_점포_수,폐업_점포_수,개폐업,개폐업_비율,유흥_총_매출_비율,패션/미용_총_매출_비율,교육_총_매출_비율,요식_총_매출_비율,의료_총_매출_비율,자동차_총_매출_비율,기타_총_매출_비율,유통업_총_매출_비율,외식_총_점포수,미용/의류_총_점포수,면적,면적당_점포수,면적당_미용의류_점포수,면적당_외식_점포수,월요일_매출_비율,화요일_매출_비율,수요일_매출_비율,목요일_매출_비율,금요일_매출_비율,토요일_매출_비율,일요일_매출_비율,남성_매출_비율,여성_매출_비율,남성_생활인구_비율,여성_생활인구_비율,연령대_10_생활인구_비율,연령대_20_생활인구_비율,연령대_30_생활인구_비율,연령대_40_생활인구_비율,연령대_50_생활인구_비율,연령대_60_이상_생활인구_비율,시간대_1_생활인구_비율,시간대_2_생활인구_비율,시간대_3_생활인구_비율,시간대_4_생활인구_비율,시간대_5_생활인구_비율,시간대_6_생활인구_비율,월요일_생활인구_비율,화요일_생활인구_비율,수요일_생활인구_비율,목요일_생활인구_비율,금요일_생활인구_비율,토요일_생활인구_비율,일요일_생활인구_비율,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,월_평균_소득_금액
0,1000001,429,-3,21,13,8,0,0.004030,0.053109,0.031199,0.575131,0.018576,0.003658,0.186805,0.127492,28.0,12.0,1.278637,335.513594,9.384996,21.898323,0.118590,0.131493,0.148730,0.132410,0.154490,0.183464,0.130823,0.386854,0.613146,0.507108,0.492892,0.124712,0.141936,0.192071,0.201521,0.154133,0.185627,0.181464,0.223587,0.170610,0.169219,0.162019,0.093100,0.149935,0.155281,0.153352,0.153040,0.149633,0.122647,0.116113,54800,0.742482,0.257518,3528699
1,1000002,218,-12,3,6,-3,2,0.013826,0.513086,0.000000,0.117492,0.000000,0.006862,0.114719,0.234014,12.0,11.0,0.323278,674.342105,34.026437,37.119749,0.116593,0.116357,0.126294,0.125684,0.156763,0.188164,0.170143,0.401264,0.598736,0.501122,0.498878,0.078217,0.164375,0.190699,0.171263,0.166074,0.229373,0.205036,0.209881,0.154166,0.154655,0.168072,0.108189,0.144139,0.145027,0.145093,0.144389,0.143292,0.139142,0.138918,47092,0.534103,0.465897,3283059
2,1000003,728,-4,11,21,-10,1,0.028540,0.065399,0.546465,0.221826,0.000324,0.010005,0.045487,0.081955,32.0,10.0,1.160775,627.167417,8.614937,27.567799,0.144447,0.144302,0.142621,0.122779,0.143191,0.211421,0.091239,0.579238,0.420762,0.572347,0.427653,0.050006,0.186137,0.187293,0.184201,0.163784,0.228580,0.130497,0.194396,0.189269,0.199620,0.203649,0.082569,0.149971,0.156542,0.155726,0.154701,0.152776,0.125837,0.104448,158024,0.662709,0.337291,2909305
3,1000004,285,-9,7,5,2,0,0.001135,0.010060,0.023782,0.418322,0.064606,0.007832,0.091438,0.382826,29.0,0.0,0.449096,634.607631,0.000000,64.574110,0.156436,0.160766,0.167509,0.139482,0.157609,0.123508,0.094691,0.549805,0.450195,0.468075,0.531924,0.170905,0.299767,0.108971,0.130250,0.109766,0.180341,0.307207,0.217697,0.109813,0.100365,0.136271,0.128645,0.143874,0.144703,0.143974,0.143808,0.140617,0.139629,0.143396,6784,0.564269,0.435731,2951442
4,1000005,152,2,2,2,0,1,0.000000,0.017641,0.011075,0.628974,0.000000,0.206369,0.074358,0.061583,20.0,5.0,3.744966,40.587820,1.335126,5.340503,0.080943,0.118395,0.140430,0.113667,0.141985,0.217003,0.187577,0.481882,0.518118,0.469972,0.530030,0.146583,0.115630,0.117679,0.176978,0.169628,0.273502,0.269451,0.218059,0.119289,0.114343,0.150837,0.128021,0.139995,0.141323,0.142361,0.140708,0.141825,0.145769,0.148019,17008,0.581373,0.418627,3977676
